In [ ]:
import torch
import clip
import os
from PIL import Image
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd

device = "cuda" if torch.cuda.is_available() else "cpu"
model, preprocess = clip.load("ViT-B/32", device=device)

In [ ]:
# image inputs
image_folder = '/Users/katharinaschmid/Text2Head/NPHM-main/rendering_data'

images = []
preprocessed_images = torch.tensor([])
file_list = sorted(os.listdir(image_folder))

keys = ['default', 'colors', 'shininess', 'light_front', '_mirror', 'antimirror', 'resolution', 'side', 'colorful']
for key in keys:
    print(key)
    for filename in file_list:
        if filename.endswith(key + ".png"):
            image = Image.open(os.path.join(image_folder, filename))
            images.append(image)
            preprocessed_image = preprocess(image).unsqueeze(0) # [1, 3, 224, 224]
            preprocessed_images = torch.cat((preprocessed_images, preprocessed_image), dim=0)
num_images = len(images)

fig, axes = plt.subplots(len(keys), 4, figsize=(15, 25))
for i, img in enumerate(images):
    ax = axes[i//4, i%4]
    ax.imshow(img)  # Display each image
    ax.axis('off')  # Turn off axis labels

plt.show()
print(preprocessed_images.shape)

# Individual Sensitivity of Parameters
Result: In general, the tests "colorful" and "antimirror" performed better than default.
A small increase in resolution doesn't lead to better results. All other tests lead to worse performance.

In [ ]:
# text inputs
general = "An image of the bust of "
captions = []
captions.append(general + "a young man")
captions.append(general + "an Asian young man")
captions.append(general + "a young man with wide forehead")
captions.append(general + "a young man with long, curly hair")
captions.append(general + "a young man with a big nose")
captions.append(general + "a chubby man")
captions.append(general + "a woman")


preprocessed_text = clip.tokenize(captions).to(device) # [num_captions, 77]

with torch.no_grad():
    image_features = model.encode_image(preprocessed_images) # [num_images, 512]
    text_features = model.encode_text(preprocessed_text) # [num_captions, 512]

# Normalize
image_features /= image_features.norm(dim=-1, keepdim=True)
text_features /= text_features.norm(dim=-1, keepdim=True)

cosine_similarity = torch.matmul(image_features, text_features.T)
cosine_similarity = cosine_similarity.reshape(-1, 4, len(captions)) # [tests, images, captions]
print(cosine_similarity.shape)

num_tests = cosine_similarity.shape[0]
print(num_tests)

for c in range(len(captions)):
    print('Test: ', captions[c])
    sim_np = cosine_similarity[:, :, c].squeeze(-1).numpy()
    row_labels = [keys[test] for test in range(num_tests)]
    col_labels = [f"Image_{i}" for i in range(4)]
    df = pd.DataFrame(sim_np, index=row_labels, columns=col_labels)
    
    # Let's analyze!
    df['Max_Image'] = df.idxmax(axis=1, numeric_only=True)
    df['Min_Image'] = df.idxmin(axis=1, numeric_only=True)
    df['Max_Value'] = df.max(axis=1, numeric_only=True)
    df['Min_Value'] = df.min(axis=1, numeric_only=True)
    
    df['Max_Delta'] = df['Max_Value'] - df['Min_Value']
    df['Max_Ratio'] = df['Min_Value'] / df['Max_Value']
    print(df)
    
    top_delta = df.nlargest(3, 'Max_Delta').index.tolist()
    top_ratio = df.nsmallest(3, 'Max_Ratio').index.tolist()
    print("Test names of the top 3 rows with highest Delta values:", top_delta)
    print("Test names of the top 3 rows with lowest Ratio values:", top_delta)

# Influence of combining image features
1) Combine all tests
--> Leads to worse performance
2) Combine top performing tests
--> Leads to a slight improvement

In [ ]:
image_features_reshaped = image_features.reshape(-1, 4, 512)

# Average over all tests
image_features_avg = image_features_reshaped.sum(dim=0)
image_features_avg = image_features_avg / image_features_avg.norm(dim=-1, keepdim=True)

cosine_similarity_avg = torch.matmul(image_features_avg, text_features.T)
print('Average over all tests')
sim_avg_np = cosine_similarity_avg.T.numpy()
row_labels = [f"Caption_{i}" for i in range(len(captions))] #[captions[test] for test in range(len(captions))]
col_labels = [f"Image_{i}" for i in range(4)]
df_avg = pd.DataFrame(sim_avg_np, index=row_labels, columns=col_labels)

# Let's analyze!
df_avg['Max_Image'] = df_avg.idxmax(axis=1, numeric_only=True)
df_avg['Min_Image'] = df_avg.idxmin(axis=1, numeric_only=True)
df_avg['Max_Value'] = df_avg.max(axis=1, numeric_only=True)
df_avg['Min_Value'] = df_avg.min(axis=1, numeric_only=True)

df_avg['Max_Delta'] = df_avg['Max_Value'] - df_avg['Min_Value']
df_avg['Max_Ratio'] = df_avg['Min_Value'] / df_avg['Max_Value']
print(df_avg)

#############
print('#########################')

# Average over top tests
image_features_top = torch.stack((image_features_reshaped[0, :, :], image_features_reshaped[5, :, :], image_features_reshaped[8, :, :]), dim=0)
image_features_avg_top = image_features_top.sum(dim=0)
image_features_avg_top = image_features_avg_top / image_features_avg_top.norm(dim=-1, keepdim=True)
cosine_similarity_avg_top = torch.matmul(image_features_avg_top, text_features.T)
print('Average over top tests')
sim_avg_top_np = cosine_similarity_avg_top.T.numpy()
row_labels = [f"Caption_{i}" for i in range(len(captions))] #[captions[test] for test in range(len(captions))]
col_labels = [f"Image_{i}" for i in range(4)]
df_avg_top = pd.DataFrame(sim_avg_top_np, index=row_labels, columns=col_labels)

# Let's analyze!
df_avg_top['Max_Image'] = df_avg_top.idxmax(axis=1, numeric_only=True)
df_avg_top['Min_Image'] = df_avg_top.idxmin(axis=1, numeric_only=True)
df_avg_top['Max_Value'] = df_avg_top.max(axis=1, numeric_only=True)
df_avg_top['Min_Value'] = df_avg_top.min(axis=1, numeric_only=True)

df_avg_top['Max_Delta'] = df_avg_top['Max_Value'] - df_avg_top['Min_Value']
df_avg_top['Max_Ratio'] = df_avg_top['Min_Value'] / df_avg_top['Max_Value']
print(df_avg_top)

# Delta analysis
CLIP(img) - CLIP(avg_img) leads to a much stronger signal!
--> antimirror + colorful better than default in all (7 and 6) cases
--> light_front, resolution and _mirror better in 3, 3, 2 cases

In [ ]:
image_features_delta = image_features_reshaped[:, 1:, :] - image_features_reshaped[:, 0, :].unsqueeze(1)

image_features_delta_norm = image_features_delta / image_features_delta.norm(dim=-1, keepdim=True)
image_features_delta_norm = image_features_delta_norm.reshape(-1, 512)
cosine_similarity_delta = torch.matmul(image_features_delta_norm, text_features.T)
cosine_similarity_delta = cosine_similarity_delta.reshape(-1, 3, len(captions))

for c in range(len(captions)):
    print('Test: ', captions[c])
    sim_delta_np = cosine_similarity_delta[:, :, c].squeeze(-1).numpy()
    row_labels = [keys[test] for test in range(num_tests)]
    col_labels = [f"Image_{i}" for i in range(3)]
    df_delta = pd.DataFrame(sim_delta_np, index=row_labels, columns=col_labels)
    
    # Let's analyze!
    df_delta['Max_Image'] = df_delta.idxmax(axis=1, numeric_only=True)
    df_delta['Min_Image'] = df_delta.idxmin(axis=1, numeric_only=True)
    df_delta['Max_Value'] = df_delta.max(axis=1, numeric_only=True)
    df_delta['Min_Value'] = df_delta.min(axis=1, numeric_only=True)
    
    df_delta['Max_Delta'] = df_delta['Max_Value'] - df_delta['Min_Value']
    df_delta['Max_Ratio'] = df_delta['Min_Value'] / df_delta['Max_Value']
    print(df_delta)
    
    top_delta = df_delta.nlargest(3, 'Max_Delta').index.tolist()
    top_ratio = df_delta.nsmallest(3, 'Max_Ratio').index.tolist()
    print("Test names of the top 3 rows with highest Delta values:", top_delta)
    print("Test names of the top 3 rows with lowest Ratio values:", top_delta)

# Text input reduced to difference in combination with Delta Analysis
--> significantly worse performance

In [ ]:
captions_diff = []
captions_diff.append("young, male")
captions_diff.append("Asian young man")
captions_diff.append("young man with wide forehead")
captions_diff.append("young man with long, curly hair")
captions_diff.append("young man with big nose")
captions_diff.append("chubby man")
captions_diff.append("woman")

preprocessed_text_diff = clip.tokenize(captions_diff).to(device)

with torch.no_grad():
    text_features_diff = model.encode_text(preprocessed_text_diff) # [num_captions, 512]

# Normalize
text_features_diff /= text_features_diff.norm(dim=-1, keepdim=True)

cosine_similarity_delta_diff = torch.matmul(image_features_delta_norm, text_features_diff.T)
cosine_similarity_delta_diff = cosine_similarity_delta_diff.reshape(-1, 3, len(captions))

for c in range(len(captions)):
    print('Test: ', captions_diff[c])
    sim_delta_diff_np = cosine_similarity_delta_diff[:, :, c].squeeze(-1).numpy()
    row_labels = [keys[test] for test in range(num_tests)]
    col_labels = [f"Image_{i}" for i in range(3)]
    df_delta_diff = pd.DataFrame(sim_delta_diff_np, index=row_labels, columns=col_labels)
    
    # Let's analyze!
    df_delta_diff['Max_Image'] = df_delta_diff.idxmax(axis=1, numeric_only=True)
    df_delta_diff['Min_Image'] = df_delta_diff.idxmin(axis=1, numeric_only=True)
    df_delta_diff['Max_Value'] = df_delta_diff.max(axis=1, numeric_only=True)
    df_delta_diff['Min_Value'] = df_delta_diff.min(axis=1, numeric_only=True)
    
    df_delta_diff['Max_Delta'] = df_delta_diff['Max_Value'] - df_delta_diff['Min_Value']
    df_delta_diff['Max_Ratio'] = df_delta_diff['Min_Value'] / df_delta_diff['Max_Value']
    print(df_delta_diff)
    
    top_delta = df_delta_diff.nlargest(3, 'Max_Delta').index.tolist()
    top_ratio = df_delta_diff.nsmallest(3, 'Max_Ratio').index.tolist()
    print("Test names of the top 3 rows with highest Delta values:", top_delta)
    print("Test names of the top 3 rows with lowest Ratio values:", top_delta)

# Delta Analysis + avg over promising tests
--> performance better than default delta but worse than antimirror, colorful

In [ ]:
# default, antimirror, colorful
image_features_delta_norm = image_features_delta / image_features_delta.norm(dim=-1, keepdim=True)

image_features_top_delta = torch.stack((image_features_delta_norm[0, :, :], image_features_delta_norm[5, :, :], image_features_delta_norm[8, :, :]), dim=0)
image_features_avg_top_delta = image_features_top_delta.sum(dim=0)
image_features_avg_top_delta = image_features_avg_top_delta / image_features_avg_top_delta.norm(dim=-1, keepdim=True)

cosine_similarity_avg_top_delta = torch.matmul(image_features_avg_top_delta, text_features.T)
print('Average over top tests')
sim_avg_top_delta_np = cosine_similarity_avg_top_delta.T.numpy()
row_labels = [f"Caption_{i}" for i in range(len(captions))] #[captions[test] for test in range(len(captions))]
col_labels = [f"Image_{i}" for i in range(3)]
df_avg_top_delta = pd.DataFrame(sim_avg_top_delta_np, index=row_labels, columns=col_labels)

# Let's analyze!
df_avg_top_delta['Max_Image'] = df_avg_top_delta.idxmax(axis=1, numeric_only=True)
df_avg_top_delta['Min_Image'] = df_avg_top_delta.idxmin(axis=1, numeric_only=True)
df_avg_top_delta['Max_Value'] = df_avg_top_delta.max(axis=1, numeric_only=True)
df_avg_top_delta['Min_Value'] = df_avg_top_delta.min(axis=1, numeric_only=True)

df_avg_top_delta['Max_Delta'] = df_avg_top_delta['Max_Value'] - df_avg_top_delta['Min_Value']
df_avg_top_delta['Max_Ratio'] = df_avg_top_delta['Min_Value'] / df_avg_top_delta['Max_Value']
print(df_avg_top_delta)

In [ ]:
# OLD CODE, NO USE FOR NOW

fig, ax = plt2.subplots()
im = ax.imshow(cosine_similarity, cmap='gray')

# Show all ticks and label them with the respective list entries
ax.set_xticks(np.arange(len(images)))
ax.set_yticks(np.arange(len(captions)), labels=captions)

# Rotate the tick labels and set their alignment.
plt2.setp(ax.get_xticklabels(), ha="right",
         rotation_mode="anchor")

# Loop over data dimensions and create text annotations.
for i in range(len(captions)):
    for j in range(len(images)):
        similarity = cosine_similarity[i, j].numpy()
        formatted_similarity = "{:.3f}".format(similarity)  # Limit to 2 decimal places
        text = ax.text(j, i, formatted_similarity,
                       ha="center", va="center", color="r")

ax.set_title("Cosine Similarity")
fig.tight_layout()
plt2.show()